In [53]:
import pandas as pd
import numpy as np

In [1]:
# compile yearly data for all products to per-product data for all years

def filename(year):
    return f'country_partner_hsproduct6digit_year_{year}.csv'

product_code = 283691

out_data = pd.DataFrame()

for year in [2015,2016,2017,2018]:
    data = pd.read_csv(f'./trade_data/HS/{filename(year)}')
    li_data = data.loc[data['hs_product_code']==product_code]
    out_data = pd.concat([out_data, li_data])
    print(f'{year} done')
    
#     li_data.to_csv(f'./trade_data/by_product/283691/{year}.csv', index=False)

out_data.to_csv(f'./trade_data/{product_code}.csv', index=False)
print('saved')


/home/unknown/anaconda3/envs/li-trade/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


2015 done
2016 done
2017 done
2018 done
saved


In [65]:
# Exploring ways to effectively capture the 'most important' nodes and links

df = pd.read_csv('./data/hs-product/283691.csv')
# print(df.columns)

top_n = 8
links_per_node = 4

top_exporters = df.groupby('location_code')['export_value'].sum().reset_index()
top_exporters = top_exporters.sort_values('export_value', ascending=False).head(n=top_n)
top_exporters_list = top_exporters['location_code'].to_list()

top_export_links = pd.melt(df, id_vars=['location_code', 'partner_code', 'year'], value_vars=['export_value'])
top_export_links = top_export_links\
    .loc[top_export_links['location_code'].isin(top_exporters_list)]\
    .sort_values('value', ascending=False)\
    .groupby(['location_code', 'year']).head(n=links_per_node)\
    .rename(columns={'value': 'export_value'})\
    .drop(columns=['variable'])

# print(top_export_links.groupby(['year', 'location_code', 'partner_code'])['export_value'].sum())

top_importers = df.groupby('location_code')['import_value'].sum().reset_index()
top_importers = top_importers.sort_values('import_value', ascending=False).head(n=top_n)
top_importers_list = top_importers['location_code'].to_list()

top_import_links = pd.melt(df, id_vars=['location_code', 'partner_code', 'year'], value_vars=['import_value'])
top_import_links = top_import_links\
    .loc[top_import_links['location_code'].isin(top_importers_list)]\
    .sort_values('value', ascending=False)\
    .groupby(['location_code', 'year']).head(n=links_per_node)\
    .rename(columns={'value': 'import_value'})\
    .drop(columns=['variable'])

top_import_links = top_import_links.rename(columns={
    'partner_code': 'tmp',
    'location_code': 'partner_code',
    'import_value': 'export_value'
}).rename(columns={'tmp': 'location_code'})

complete_links = top_import_links.merge(top_export_links, how='outer', on=['location_code', 'partner_code', 'year'], indicator=True)
complete_links['value'] = complete_links['export_value_x'].fillna(complete_links['export_value_y'])
complete_links = complete_links[['year', 'location_code', 'partner_code', 'value', '_merge']].sort_values('value', ascending=False)
print(complete_links)





     year location_code partner_code        value      _merge
0    2018           CHL          KOR  339958272.0        both
1    2018           CHL          JPN  223374768.0        both
2    2017           CHL          KOR  205031424.0        both
3    2017           CHL          CHN  177681232.0        both
4    2018           ARG          CHN  173507088.0        both
..    ...           ...          ...          ...         ...
125  2016           IND          USA     149027.0   left_only
126  2015           CHN          RUS     144105.0   left_only
180  2016           KOR          IDN     134039.0  right_only
127  2017           GBR          USA     117921.0   left_only
181  2015           KOR          BGD      12297.0  right_only

[182 rows x 5 columns]


In [ ]:
'''
top_imports = df.loc[
    (df['location_code'].isin(im_top['location_code'].to_list())) &
    (df['partner_code'].isin(ex_top['location_code'].to_list())),
['location_code', 'partner_code', 'year', 'import_value']]

##
## Top xxports provides the link-level data (volume per bilateral pair per year)
## while pivoting total volumes per country by year prepares the node-level data
## to be broken down into a dictionary of {year: volume} values.
##

top_imports = top_imports.rename(columns={
    'location_code': 'source',
    'partner_code': 'target',
    'import_value': 'volume'
})

top_importers = top_imports.groupby(['source', 'year'])['volume'].sum().reset_index()
top_importers_volumes = top_importers.pivot(index='source', columns='year', values='volume').to_dict('index')


top_exports = df.loc[
    (df['location_code'].isin(ex_top['location_code'].to_list())) &
    (df['partner_code'].isin(im_top['location_code'].to_list())),
['location_code', 'partner_code', 'year', 'export_value']]

top_exports = top_exports.rename(columns={
    'location_code': 'source',
    'partner_code': 'target',
    'export_value': 'volume'
})

top_exporters = top_exports.groupby(['source', 'year'])['volume'].sum().reset_index()
top_exporters_volumes = top_exporters.pivot(index='source', columns='year', values='volume').to_dict('index')

##
## Given a country c, return year-by-year volumes per partner country
##
def get_trade_links(country, flowtype):
    if flowtype=='export':
        flow_data = top_exports.loc[top_exports['source']==country]
    elif flowtype=='import':
        flow_data = top_imports.loc[top_imports['source']==country]
    else:
        raise('Bad flowtype in get_trade_links')
        
    partner_data = flow_data.groupby(['target', 'year'])['volume'].sum().reset_index()
    partner_data = partner_data.pivot(index='target', columns='year', values='volume')
    partner_data = partner_data.fillna(0).astype(int)
#     print(country)
#     print(partner_data)
#     print()
    return partner_data.to_dict('index')


## 
## Final output data should look like:
## {
##   nodes: [ {name: n, lat: y, lon: x, volume: {year: v, ...}, ... ],
##   links: [ {source: n1, target: n2, volume: {year: v, ...}, ... ]
## }
## 
## 


output_nodes = []
output_links = []

country_codes = pd.read_csv('./country_codes.csv').set_index('alpha3')

for country in top_countries:
    ##
    ## Some countries may be both top importers and top exporters, so rather
    ## than type={import, export}, use type=[0,1] where 0 = pure importer
    ##
    if country in top_importers_volumes.keys():
        # Each bilateral link only needs to be included once - because of the
        # way the if-else is structured below, it's easiest to count links at
        # the import node, to catch trade with both 'pure' and 'mixed' nodes
        
        import_links = get_trade_links(country, 'import')
        output_links.extend([
            {
                "source": partner,
                "target": country,
                "volume": volumes
            }
            for partner, volumes in import_links.items()
        ])
        
        if country in top_exporters_volumes.keys():
            # Then this country has both inflows and outflows to account for
            total_volume = {
                i: v + top_exporters_volumes[country][i]
                for i,v in top_importers_volumes[country].items()
            }
            vol_mixture = {
                i: round(v / total_volume[i],3)
                for i,v in top_importers_volumes[country].items()
            }
            output_nodes.append({
                "name": country,
                "lat": country_codes.loc[country, 'latitude'],
                "lon": country_codes.loc[country, 'longitude'],
                "type": vol_mixture,
                "volume": total_volume,
            })
        else:
            output_nodes.append({
                "name": country,
                "lat": country_codes.loc[country, 'latitude'],
                "lon": country_codes.loc[country, 'longitude'],
                "type": {i: 0 for i in top_importers_volumes[country].keys()},
                "volume": top_importers_volumes[country],
            })
    else:
        output_nodes.append({
            "name": country,
            "lat": country_codes.loc[country, 'latitude'],
            "lon": country_codes.loc[country, 'longitude'],
            "type": {i: 1 for i in top_exporters_volumes[country].keys()},
            "volume": top_exporters_volumes[country],
        })

print()
print(output_nodes)
print()
print(output_links)

output_data = {
    "nodes": output_nodes,
    "links": output_links
}


#.apply(lambda x: x[['year', 'import_value']].to_dict('records')).reset_index().to_json(orient='records'))


# top_exports = df.loc[df['location_code'].isin(ex_top['location_code'].to_list())]

# print(top_imports)

# df = df.loc[
#     (df['location_code'].isin(top_countries)) &
#     (df['partner_code'].isin(top_countries))
# , ['location_code', 'year', 'import_value', 'export_value']]

# print(df)
'''

In [12]:
import requests
import json

# response = requests.get('https://bm849xmvuf.execute-api.ap-southeast-2.amazonaws.com/testing/98765')
response = requests.get('https://frklvrq4cj.execute-api.ap-southeast-2.amazonaws.com/testing/product?hs_code=666')
print(response.status_code)
print(json.loads(response.content))

200
{'statusCode': 200, 'body': '{"nodes": [{"name": "ARG", "lat": -38.416097, "lon": -63.616672, "type": {"2015": 1, "2016": 1, "2017": 1, "2018": 1}, "volume": {"2015": 51938636, "2016": 149613703, "2017": 209709059, "2018": 280178591}}, {"name": "BEL", "lat": 50.503887, "lon": 4.469936, "type": {"2015": 0.664, "2016": 0.675, "2017": 0.637, "2018": 0.652}, "volume": {"2015": 50503037, "2016": 132301623, "2017": 153401480, "2018": 218545198}}, {"name": "CHL", "lat": -35.675147, "lon": -71.542969, "type": {"2015": 1, "2016": 1, "2017": 1, "2018": 1}, "volume": {"2015": 220765987, "2016": 443890512, "2017": 649963871, "2018": 933606055}}, {"name": "CHN", "lat": 35.86166, "lon": 104.195397, "type": {"2015": 0.862, "2016": 0.852, "2017": 0.905, "2018": 0.67}, "volume": {"2015": 57746196, "2016": 155053323, "2017": 287855379, "2018": 463619796}}, {"name": "DEU", "lat": 51.165691, "lon": 10.451526, "type": {"2015": 0.647, "2016": 0.695, "2017": 0.637, "2018": 0.618}, "volume": {"2015": 3755